In [31]:
from datetime import date, datetime, timedelta
import pandas as pd

import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn import metrics, preprocessing
from sklearn.model_selection import KFold

from sklearn.decomposition import PCA
from scipy import stats
import seaborn as sns

import os
from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
def obtain_intervals(dataset):
    '''
    Generate interval terminals, so that samples in each interval have:
        interval_i = (timestamp >= terminal_i) and (timestamp < terminal_{i+1})

    Args:
        dataset (chr): Assuming only Backblaze (b) and Google (g) datasets exists
    '''
    if dataset == 'g':
        # time unit in Google: millisecond, tracing time: 29 days
        start_time = 604046279
        unit_period = 24 * 60 * 60 * 1000 * 1000  # unit period: one day
        end_time = start_time + 28*unit_period
    elif dataset == 'b':
        # time unit in Backblaze: month, tracing time: one year (12 months)
        start_time = 1
        unit_period = 1  # unit period: one month
        end_time = start_time + 12*unit_period

    # add one unit for the open-end of range function
    terminals = [i for i in range(start_time, end_time+unit_period, unit_period)]

    return terminals

In [34]:
def obtain_natural_chunks(features, labels, terminals):
    feature_list = []
    label_list = []
    for i in range(len(terminals) - 1):
        idx = np.logical_and(features[:, 0] >= terminals[i], features[:, 0] < terminals[i + 1])
        feature_list.append(features[idx][:, 1:])
        label_list.append(labels[idx])
    return feature_list, label_list


In [35]:
def sorting_features_by_importance(df_feature_importance, features_name):
    # Extract Ranks
    feature_imp_array_string = np.array(((df_feature_importance.FI[period].replace('[','')).replace(']','')).replace('\n', '').split(' '))
    #print('feature_imp_array_string', feature_imp_array_string)

    # convert to float
    feature_imp_array = [float(i) for i in feature_imp_array_string if i != '' ]

    # consider only the most important features (importance > mean(feature_importances))
    # extract mean?? of feature importance
    mean_importance = np.mean(feature_imp_array)

    zipped_features = list(zip(feature_imp_array, features_name))


    sorted_features_zip = sorted(zipped_features, key = lambda x: x[0], reverse = True)

   
    return sorted_features_zip, mean_importance

In [36]:
def KS_on_features(df_train, df_test):
    stat_test = stats.kstest
    distribution_training = sns.distplot(np.array(df_train)).get_lines()[0].get_data()[1]
    plt.close()
    distribution_test = sns.distplot(np.array(df_test)).get_lines()[0].get_data()[1]
    plt.close()
    v, p = stat_test(distribution_training, distribution_test)
    if(p<0.05):
        return 1
    else:
        return 0

In [37]:
def features_labels_preprocessing(DATASET_PATH, dataset):
    
    if(dataset=='b'):
        
        print('Data Reading and Preprocessing')
        
        # set data paths and columns names
        features_disk_failure = ['smart_1_raw', 'smart_4_raw', 'smart_5_raw', 'smart_7_raw', 'smart_9_raw', 'smart_12_raw', 'smart_187_raw', 'smart_193_raw', 'smart_194_raw', 'smart_197_raw', 'smart_199_raw', 
                         'smart_4_raw_diff', 'smart_5_raw_diff', 'smart_9_raw_diff', 'smart_12_raw_diff', 'smart_187_raw_diff', 'smart_193_raw_diff', 'smart_197_raw_diff', 'smart_199_raw_diff']
        columns = ['serial_number', 'date'] + features_disk_failure + ['label']
        
        # read dataset
        df = pd.read_csv(DATASET_PATH, header=None)
        df.columns = columns
        # ignore serial number
        df = df[df.columns[1:]]
        
        # transform date to date time
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
        
        # divide on weeks
        df['date'] = pd.Series(pd.DatetimeIndex(df['date']).day_of_year)
        
        print('Features and Labels Computing')
        
        # features and labels extraction and computation
        features = df[df.columns[:-1]].to_numpy()
        labels = df[df.columns[-1]].to_numpy()
        feature_list, label_list = obtain_natural_chunks(features, labels, obtain_intervals('b'))
        
    elif(dataset=='g'):
        
        print('Data Reading and Preprocessing')
        
        # set data paths and columns names
        features_job_failure = ['User ID', 'Job Name', 'Scheduling Class',
                   'Num Tasks', 'Priority', 'Diff Machine', 'CPU Requested', 'Mem Requested', 'Disk Requested',
                   'Avg CPU', 'Avg Mem', 'Avg Disk', 'Std CPU', 'Std Mem', 'Std Disk']
        columns_initial = ['Job ID', 'Status', 'Start Time', 'End Time'] + features_job_failure
        
        # read dataset
        df = pd.read_csv(DATASET_PATH)
        #df.columns = columns_initial
        df = df.tail(-1)
        # ignore Job ID
        df = df.drop(['Job ID'], axis = 1)
        columns = features_job_failure

        include_end_time = False
        
        print('Features and Labels Preprocessing')
        
        # features and labels preprocessing
        features = df[(['Start Time']+ features_job_failure)].to_numpy()
        labels = (df['Status']==3).to_numpy()

        # FEATURES PREPROCESSING
        offset = (1 if include_end_time else 0)

        # ENCODE USER ID
        le = preprocessing.LabelEncoder()
        features[:, 1+offset] = le.fit_transform(features[:, 1+offset])

        # ENCODE JOB NAME
        le = preprocessing.LabelEncoder()
        features[:, 2+offset] = le.fit_transform(features[:, 2+offset])

        features = features.astype(float)
        
        print('Features and Labels Computing')
        
        # features and labels extraction and computation
        feature_list, label_list = obtain_natural_chunks(features, labels, obtain_intervals('g'))
        
    else:
        print('Incorrect Dataset')
    
    return feature_list, label_list

In [38]:
DATASET_PATH = '../../../Documents/phd_related/data_sets_concept_drift/AIOps_failure_prediction/google_job_failure.csv'
interval = 'd'


In [39]:
feature_list, label_list = features_labels_preprocessing(DATASET_PATH, 'g')

Data Reading and Preprocessing
Features and Labels Preprocessing
Features and Labels Computing


In [40]:
data_path_drift_localization = '../../../Documents/phd_related/alibaba/results/results_r/rf_concept_drift_localization_job_google_r_avg.csv'
random_seeds = ['1234', '4887', '597', '1959', '413', '44', '2969', '4971', '4913', '9591']

In [41]:
df_concept_drift = pd.read_csv(data_path_drift_localization)
df_concept_drift = df_concept_drift.loc[:, ~df_concept_drift.columns.str.contains('^Unnamed')]
df_concept_drift

,X,Sig,Y,P,FI,Dataset,Model
0,2,True,inf,D1_2,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],Google,Random Forests
1,3,True,2.251150,D2_3,[1.40656093e-01 6.77843712e-02 2.38335246e-02 ...,Google,Random Forests
2,4,False,0.183823,D3_4,[0.09965596 0.06313577 0.01863725 0.07117918 0...,Google,Random Forests
3,5,False,0.038504,D4_5,[0.07262888 0.08244205 0.03470399 0.02823718 0...,Google,Random Forests
4,6,True,0.231693,D5_6,[0.1373757 0.06440213 0.02295284 0.04328491 0...,Google,Random Forests
...,...,...,...,...,...,...,...
265,267,False,0.132754,D25_26,[0.11301451 0.07830951 0.03246196 0.04946277 0...,Google,Random Forests
266,268,False,0.185512,D26_27,[0.10309081 0.0719297 0.02810484 0.03886336 0...,Google,Random Forests
267,269,False,0.040528,D27_28,[0.09824896 0.06445325 0.02718747 0.03470019 0...,Google,Random Forests
268,270,True,0.277025,D28_29,[0.09174494 0.05765307 0.03657926 0.04957106 0...,Google,Random Forests


In [42]:
df_concept_drift[df_concept_drift.P == 'D11_12']

,X,Sig,Y,P,FI,Dataset,Model
10,12,False,0.132810,D11_12,[0.11471472 0.06489837 0.03771919 0.05409328 0...,Google,Random Forests
37,39,False,0.127868,D11_12,[0.10864288 0.06645777 0.03607146 0.05281076 0...,Google,Random Forests
64,66,False,0.090648,D11_12,[0.11109902 0.07531936 0.04025066 0.04276139 0...,Google,Random Forests
91,93,False,0.124602,D11_12,[0.11854283 0.06391657 0.03020959 0.05215946 0...,Google,Random Forests
118,120,False,0.111866,D11_12,[0.11561881 0.06474925 0.03841227 0.05422171 0...,Google,Random Forests
145,147,False,0.115662,D11_12,[0.11985708 0.07247942 0.03367332 0.04859606 0...,Google,Random Forests
172,174,False,0.224744,D11_12,[0.12498786 0.06309168 0.03772603 0.06141843 0...,Google,Random Forests
199,201,False,0.237429,D11_12,[0.13111437 0.06739682 0.03377451 0.05873947 0...,Google,Random Forests
226,228,False,0.173848,D11_12,[0.12922711 0.0631992 0.03272217 0.0654309 0...,Google,Random Forests
253,255,False,0.218024,D11_12,[0.13485554 0.0599576 0.03627941 0.05516142 0...,Google,Random Forests


In [43]:
features_job_failure = ['User ID', 'Job Name', 'Scheduling Class',
                   'Num Tasks', 'Priority', 'Diff Machine', 'CPU Requested', 'Mem Requested', 'Disk Requested',
                   'Avg CPU', 'Avg Mem', 'Avg Disk', 'Std CPU', 'Std Mem', 'Std Disk']

In [44]:
no_overall_correct_all_total = []
no_overall_correct_important_total = []
no_overall_correct_pca_total = []

no_drift_correct_all_total = []
no_drift_correct_important_total = []
no_drift_correct_pca_total = []

no_non_drift_correct_all_total = []
no_non_drift_correct_important_total = []
no_non_drift_correct_pca_total = []


random_seed_all = []



drifts_true_job = []
for k in range(0, len(df_concept_drift.Sig.values)):
    if(df_concept_drift.Sig.values[k]==True):
        drifts_true_job.append(1)
    else:
        drifts_true_job.append(0)

scaler = StandardScaler()


ks_results_all_features = []
ks_results_important_features = []
ks_results_pca_features = []


for period in tqdm(range(0, len(feature_list)-1)):
#for period in tqdm(range(1,2)):


    #print('Before Scale Train', feature_list[period])
    #print('Before Scale Test', feature_list[period+1])

    # extract features train and test
    training_features = scaler.fit_transform(feature_list[period])
    testing_features = scaler.transform(feature_list[period+1])

    # convert numpy array to Pandas Dataframe
    df_train_features = pd.DataFrame(training_features, columns = features_job_failure)
    df_test_features = pd.DataFrame(testing_features, columns = features_job_failure)

    # Sort by Feature Importance to avoid         
    sorted_features_zip, mean_importance = sorting_features_by_importance(df_concept_drift, features_job_failure)



    # Using All Features
    sorted_features_all = [i[1] for i in sorted_features_zip]
    #print('length sorted_features All', len(sorted_features_all))
    df_train_features_sorted_all = df_train_features[sorted_features_all]
    df_test_features_sorted_all = df_test_features[sorted_features_all]

    # Using Most Important Features

    sorted_important_features_filter = [x for x in sorted_features_zip if x[0]>=mean_importance]
    sorted_features_important = [i[1] for i in sorted_important_features_filter]
    #print('length sorted_features Important', len(sorted_features_important))

    df_train_features_sorted_important = df_train_features[sorted_features_important]
    df_test_features_sorted_important = df_test_features[sorted_features_important]

    # Using PCA on Features
    # reduce features dimensionality using PCA
    pca = PCA(n_components = 0.95)
    pca.fit(df_train_features_sorted_all)

    df_train_features_sorted_pca = pca.transform(df_train_features_sorted_all)
    df_test_features_sorted_pca = pca.transform(df_test_features_sorted_all)


    ks_results_all_features.append(KS_on_features(df_train_features_sorted_all, df_test_features_sorted_all))
    ks_results_important_features.append(KS_on_features(df_train_features_sorted_important, df_test_features_sorted_important))
    ks_results_pca_features.append(KS_on_features(df_train_features_sorted_pca, df_test_features_sorted_pca))


print('Results All Features', ks_results_all_features)
print('Results Important Features', ks_results_important_features)
print('Results PCA Features', ks_results_pca_features)


100%|██████████| 27/27 [01:26<00:00,  3.19s/it]

Results All Features [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Results Important Features [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
Results PCA Features [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1]


In [48]:
print(ks_results_all_features)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [49]:
print(ks_results_important_features)

[1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]


In [50]:
print(ks_results_pca_features)

[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1]


In [16]:
len(ks_results_all_features)

27

In [17]:
len(ks_results_important_features)

27

In [18]:
len(ks_results_pca_features)

27

In [19]:
len(ks_results_all_features * 10)

270

In [20]:
ks_results_all_features_all_rs = ks_results_all_features * 10
ks_results_important_features_all_rs = ks_results_important_features * 10
ks_results_pca_features_all_rs = ks_results_pca_features * 10

In [21]:


no_overall_correct_all = 0
no_drift_correct_all = 0
no_non_drift_correct_all = 0


no_overall_correct_important = 0
no_drift_correct_important = 0
no_non_drift_correct_important = 0

no_overall_correct_pca = 0
no_drift_correct_pca = 0
no_non_drift_correct_pca = 0

for j in tqdm(range(0, len(ks_results_all_features_all_rs))):

    # Overall correct all 3 cases
    if(ks_results_all_features_all_rs[j]==drifts_true_job[j]):
        no_overall_correct_all = no_overall_correct_all + 1

    if(ks_results_important_features_all_rs[j]==drifts_true_job[j]):
        no_overall_correct_important = no_overall_correct_important + 1

    if(ks_results_pca_features_all_rs[j]==drifts_true_job[j]):
        no_overall_correct_pca = no_overall_correct_pca + 1

    # Correctly Identified Drifts all 3 cases

    # All Features
    if(drifts_true_job[j]==1):
        if(ks_results_all_features_all_rs[j]==drifts_true_job[j]):
            no_drift_correct_all = no_drift_correct_all + 1

    # Most Important Features
    if(drifts_true_job[j]==1):
        if(ks_results_important_features_all_rs[j]==drifts_true_job[j]):
            no_drift_correct_important = no_drift_correct_important + 1

    # PCA on Features
    if(drifts_true_job[j]==1):
        if(ks_results_pca_features_all_rs[j]==drifts_true_job[j]):
            no_drift_correct_pca = no_drift_correct_pca + 1

     # Correctly Identified Non-Drift all 3 cases

    # All Features
    if(drifts_true_job[j]==0):
        if(ks_results_all_features_all_rs[j]==drifts_true_job[j]):
            no_non_drift_correct_all = no_non_drift_correct_all + 1

    # Most Important Features
    if(drifts_true_job[j]==0):
        if(ks_results_important_features_all_rs[j]==drifts_true_job[j]):
            no_non_drift_correct_important = no_non_drift_correct_important + 1

    # PCA on Features
    if(drifts_true_job[j]==0):
        if(ks_results_pca_features_all_rs[j]==drifts_true_job[j]):
            no_non_drift_correct_pca = no_non_drift_correct_pca + 1

# Compute Metrics for all Random Seeds

# Overall Correct Predictions
overall_correct_prediction_score_all = no_overall_correct_all/len(drifts_true_job)
overall_correct_prediction_score_important = no_drift_correct_important/len(drifts_true_job)
overall_correct_prediction_score_pca = no_overall_correct_pca/len(drifts_true_job)

no_overall_correct_all_total.append(overall_correct_prediction_score_all)
no_overall_correct_important_total.append(overall_correct_prediction_score_important)
no_overall_correct_pca_total.append(overall_correct_prediction_score_pca)

# Correctly Identified Drifts

drift_correct_prediction_score_all = no_drift_correct_all/len(np.nonzero(drifts_true_job)[0])
drift_correct_prediction_score_important = no_drift_correct_important/len(np.nonzero(drifts_true_job)[0])
drift_correct_prediction_score_pca = no_drift_correct_pca/len(np.nonzero(drifts_true_job)[0])

no_drift_correct_all_total.append(drift_correct_prediction_score_all)
no_drift_correct_important_total.append(drift_correct_prediction_score_important)
no_drift_correct_pca_total.append(drift_correct_prediction_score_pca)


# Correctly Identified Non-Drifts

non_drift_correct_prediction_score_all = no_non_drift_correct_all/(len(drifts_true_job) - len(np.nonzero(drifts_true_job)[0]))
non_drift_correct_prediction_score_important = no_non_drift_correct_important/(len(drifts_true_job) - len(np.nonzero(drifts_true_job)[0]))
non_drift_correct_prediction_score_pca = no_non_drift_correct_pca/(len(drifts_true_job) - len(np.nonzero(drifts_true_job)[0]))


no_non_drift_correct_all_total.append(non_drift_correct_prediction_score_all)
no_non_drift_correct_important_total.append(non_drift_correct_prediction_score_important)
no_non_drift_correct_pca_total.append(non_drift_correct_prediction_score_pca)

#random_seed_all.append([random_seeds[i-1]]*len(feature_list))


100%|██████████| 270/270 [00:00<00:00, 561458.64it/s]


### Results All Features

In [22]:
np.mean(no_overall_correct_all_total)

0.6296296296296297

In [23]:
np.mean(no_drift_correct_all_total)

0.9411764705882353

In [24]:
np.mean(no_non_drift_correct_all_total)

0.1

### Results Important Features

In [25]:
np.mean(no_overall_correct_important_total)

0.5185185185185185

In [26]:
np.mean(no_drift_correct_important_total)

0.8235294117647058

In [27]:
np.mean(no_non_drift_correct_important_total)

0.0

### Results PCA Features

In [28]:
np.mean(no_overall_correct_pca_total)

0.5185185185185185

In [29]:
np.mean(no_drift_correct_pca_total)

0.6470588235294118

In [30]:
np.mean(no_non_drift_correct_pca_total)

0.3